In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

site = "Stevens Pass"
sites = [
    "Stevens Pass",
    "Mt. Baker Ski Area",
    "Snoqualmie Pass",
    "White Pass",
    "HURW1"
]

options = webdriver.ChromeOptions()
options.add_experimental_option("prefs", {
    "download.default_directory": r"C:\Users\dlhogan\OneDrive - UW\Documents\GitHub\Cascski-Alpine-Insights\cascade-mountain-weather.github.io\data\forecasts",
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True,
    "profile.default_content_setting_values.automatic_downloads": 1
})

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

driver.get(
    f"https://apps.gsl.noaa.gov/nbmviewer/?col=2&hgt=1&obs=false&fontsize=1&location={site.replace(' ','+')}&selectedgroup=Observed&darkmode=on&graph=fa-chart-bar&probfield=Tmax&proboperator=%3E%3D&probvalue=40&colorfriendly=false&whiskers=false&boxes=true&median=false&det=true&tz=local"
)

time.sleep(2)

# Locate the download button
button = driver.find_element(By.CSS_SELECTOR, "button.fa-download")
button.click()

# Wait for download to finish
time.sleep(8)

driver.quit()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
# os.chdir("..")
working_dir = os.getcwd()
col_df = pd.read_csv(os.path.join(working_dir, 'data', 'forecasts', 'nbm_cols.csv'))
cols = ['ValidTime',
        'ASNOW72hr_surface_25% level',
        'ASNOW72hr_surface_50% level',
        'ASNOW72hr_surface_75% level', 
        "ASNOW72hr_surface",
        'ASNOW1hr_surface_25% level',
        'ASNOW1hr_surface_50% level',
        'ASNOW1hr_surface_75% level', 
        "ASNOW1hr_surface",
        'ASNOW6hr_surface_25% level',
        'ASNOW6hr_surface_50% level',
        'ASNOW6hr_surface_75% level', 
        "ASNOW6hr_surface",
        'ASNOW24hr_surface_25% level',
        'ASNOW24hr_surface_50% level',
        'ASNOW24hr_surface_75% level', 
        "ASNOW24hr_surface"]

nbm_df = pd.read_csv(os.path.join(working_dir, 'data', 'forecasts', 'Stevens Pass (1).csv'))
nbm_df = nbm_df[cols]
nbm_df['ValidTime'] = pd.to_datetime(nbm_df['ValidTime'], format="%Y%m%d%H")
nbm_df = nbm_df.set_index('ValidTime')

In [ ]:
import json

if site == "HURW1":
    site = "Hurricane Ridge"
dates = slice(nbm_df.index[0], nbm_df.index[0] + pd.Timedelta(hours=72))
with open(os.path.join(working_dir, 'data', 'forecasts', 'forecast_2025-11-27.json'), 'r') as f:
    forecast_data = json.load(f)

In [ ]:
forecast_data['areas'][site]['nbm_forecast']['deterministic'] = (nbm_df.loc[dates, 'ASNOW6hr_surface']*4*10).sum()
forecast_data['areas'][site]['nbm_forecast']['range'] = [(nbm_df.loc[dates, 'ASNOW6hr_surface_25% level']*4*10).sum(),
                                                            (nbm_df.loc[dates, 'ASNOW6hr_surface_75% level']*4*10).sum()]

In [111]:
# write to file
with open(os.path.join(working_dir, 'data', 'forecasts', 'forecast_2025-11-27.json'), 'w') as f:
    json.dump(forecast_data, f, indent=4)